In [61]:
import pandas as pd
import numpy as np
from sklearn import neighbors
import matplotlib.pyplot as plt
from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH


# Cargar set de datos

In [2]:
trip = pd.read_csv('data/trip_train.csv')

In [3]:
weather = pd.read_csv('data/weather.csv')

In [4]:
station = pd.read_csv('data/station.csv')

# Modificando set de datos

In [5]:
stationReduced = station.loc[:,['id','name','city']]

def choseZipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

stationReduced['zip_code'] = stationReduced.loc[:,'city'].apply(choseZipCode)

stationReduced = stationReduced.drop(labels=['id','city'],axis=1)
stationReduced.rename(columns={'name':'station'},inplace=True)
trip.rename(columns={'start_station_name':'station'},inplace=True)

In [6]:
trip = trip.drop(labels=['zip_code','end_date','end_station_name','end_station_id','bike_id'],axis=1)

In [7]:
weather['precipitation_inches'] = weather['precipitation_inches'].apply(lambda x : 0.005 if (x == 'T') else float(x))

In [8]:

weather = weather[weather.precipitation_inches.isnull() == False]

weather = weather.fillna(0)

weather['events'] = weather['events'].apply(lambda x : 'Normal' if x == 0 else x)
weather['events'] = weather['events'].apply(lambda x : 'Rain' if x == 'rain' else x)
weather['date'] = weather['date'].apply(lambda date : pd.to_datetime(date).date())

In [9]:
def splitDateAndTime(dateAndTime):
    splitLine = dateAndTime.split(' ')
    return str(splitDate(splitLine[0]))  + splitTime(splitLine[1])

def splitDate(date):
    lineSplit = date.split('/')
    return str(lineSplit[2]) + str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1] if int(lineSplit[1]) >= 10 else '0' + str(lineSplit[1]) )

def splitTime(time):
    lineSplit = time.split(':')
    return str(lineSplit[0] if int(lineSplit[0]) >= 10 else '0' + str(lineSplit[0])) + str(lineSplit[1])


In [10]:
trip['time'] = trip['start_date'].apply(lambda x : int(splitDateAndTime(x)[8:12]) )

In [11]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(splitDateAndTime(x)))

In [12]:
trip.loc[:,'day_of_week'] = trip['start_date'].apply(lambda x : x.dayofweek )

In [13]:
trip.loc[:,'month'] = trip['start_date'].apply(lambda x : x.month )

In [14]:
trip.loc[:,'year'] = trip['start_date'].apply(lambda x : x.year )

In [15]:
trip.loc[:,'dayofyear'] = trip['start_date'].apply(lambda x : x.dayofyear )

In [16]:
trip.loc[:,'start_date'] = trip['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [17]:
cal = calendarUSFH()
holidays = cal.holidays(return_name=True,start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))
holiday_festive_day = pd.DataFrame(holidays,columns=['holiday']).reset_index()
holiday_festive_day.rename(columns={'index':'start_date'},inplace=True)

In [18]:
holiday_festive_day.loc[:,'start_date'] = holiday_festive_day['start_date'].apply(lambda x : pd.to_datetime(x).date())

In [19]:
trip = pd.merge(holiday_festive_day,trip,on=['start_date'],how='right')

In [20]:
trip.loc[:,'is_holiday'] = trip.holiday.isnull()

In [21]:
trip.loc[:,'is_holiday'] = trip['is_holiday'].apply(lambda x : not x )

In [22]:
trip.loc[:,'subscription_type'] = trip['subscription_type'].apply(lambda x : 0 if ('Subscriber' == x) else 1 )

In [23]:
trip.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday
255391,2015-07-14,NaN,846039,96,Market at Sansome,77,0,1436,1,7,2015,195,False


In [24]:
tripWithZipCode = pd.merge(trip,stationReduced,on=['station'],how='right')

In [25]:

tripWithZipCode.sample()

,start_date,holiday,id,duration,station,start_station_id,subscription_type,time,day_of_week,month,year,dayofyear,is_holiday,zip_code
336439,2015-07-17,NaN,850553,657,San Francisco Caltrain (Townsend at 4th),70,0,746,4,7,2015,198,False,94107


In [26]:
tripWithZipCode = tripWithZipCode.drop(labels=['holiday'],axis=1).dropna()

In [27]:
tripWithZipCode.rename(columns={'start_date':'date'},inplace=True)

In [28]:
tripsWithWeather = pd.merge(tripWithZipCode,weather,on=['date','zip_code'],how='left')

In [29]:
tripsWithWeather.rename(columns={'start_station_id':'station_id'},inplace=True)

In [30]:
tripsWithWeather = tripsWithWeather.dropna()

In [31]:
tripsWithWeather = tripsWithWeather[tripsWithWeather['duration'] < 1000]

In [32]:
tripsWithWeather.shape

(469476, 35)

In [33]:
test_target = tripsWithWeather['duration'][-50000:]

In [34]:
train_target = tripsWithWeather['duration'][0:tripsWithWeather.shape[0]-50001]

In [35]:
tripsWithWeatherReduced = tripsWithWeather.drop(labels=['duration','date','station','events'],axis=1)

In [36]:
from sklearn.decomposition import IncrementalPCA

In [37]:
pca = IncrementalPCA(n_components=8)

In [38]:
tripTransform = pca.fit_transform(tripsWithWeatherReduced,)

In [125]:
tripTransform.shape

(469476, 8)

In [126]:
tripTransformDF = pd.DataFrame(tripTransform)

In [127]:
tripTransformDF = tripTransformDF.abs()

In [128]:
train = tripTransformDF.iloc[0:tripsWithWeather.shape[0]-50001]

In [129]:
test = tripTransformDF.iloc[-50000:]

In [130]:
train.shape

(419475, 8)

In [131]:
train_target.shape

(419475,)

# Machine Learning

In [132]:
from sklearn.neural_network import MLPRegressor

In [133]:
mlpr = MLPRegressor()

In [134]:
mlpr.fit(train,train_target)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [135]:
test_prediction = mlpr.predict(test)

In [136]:
mlpr.score(test,test_target)

-0.7376135014136207

In [116]:
predictionDF = pd.DataFrame(test_prediction,columns={"prediction"})

In [117]:
predictionDF.loc[:,'spectate'] = test_target.values

In [118]:

predictionDF.sample(5)

,prediction,spectate
32577,384.838453,304
32011,459.875568,921
13403,271.782390,175
29792,328.247164,931
27530,268.925769,184


In [119]:
def predictError(predict,spectate):
    if (spectate > predict) : return  ((spectate-predict)/spectate)*100
    else  :  return ((predict-spectate)/predict )*100

In [120]:
predictionDF['errorValue'] = predictionDF.apply(axis=1,func=lambda x : predictError(x[0],x[1]))

In [121]:
predictionDF.sample(5)

,prediction,spectate,errorValue
21125,347.017652,690,49.707587
22192,397.364559,650,38.866991
48650,257.879290,702,63.265058
41627,297.393591,110,63.011980
19596,248.197588,296,16.149463


In [122]:
print 'El error promedio que comete KNN al predecir es de', float(predictionDF['errorValue'].mean()),'%'

El error promedio que comete KNN al predecir es de 35.6246291849 %


In [123]:
print "El algoritmo Neuronal Networks predice un ",\
float(predictionDF[predictionDF['errorValue'] < 15].shape[0]) / float(predictionDF.shape[0]) *100 ,\
"% de los valores, con un error menor al 15%"


El algoritmo Neuronal Networks predice un  18.278 % de los valores, con un error menor al 15%
